1) Реализовать простейший динамический массив, поддерживающий добавление элемента в конец массива при помощи метода append(e).
Также должны поддерживаться следующие операции:
-  извлечение элемента по индексу;
-  устанавка нового значения по индексу;
-  получение текущей длины массива.
1.1) Добавить реализацию удаления элемента из любого места массива и поддержку корректного вывода массива при помощи функции print.

In [1]:
class DynamicArray:
    def __init__(self):
        self.data = []

    def append(self, e):
        self.data.append(e)

    def get_element(self, index):
        return self.data[index]

    def set_element(self, index, value):
        self.data[index] = value

    def delete_element(self, index):
        del self.data[index]

    def get_length(self):
        return len(self.data)

    def __str__(self):
        return str(self.data)


# Пример использования
arr = DynamicArray()
arr.append(1)
arr.append(2)
arr.append(3)
print(arr.get_length())
print(arr.get_element(1))
arr.set_element(1, 5)
print(arr)
arr.delete_element(1)
print(arr)



3
2
[1, 5, 3]
[1, 3]


2)Реализовать генератор, который возвращает значение поочередно извлекаемое из конца двух очередей (в качестве очереди используется deque из collections). Если очередь из которой извлекается элемент пуста - генератор заканчивает работу.

In [3]:
from collections import deque

def deque_value_generator(queue1, queue2):
    while queue1 or queue2:
        if queue1:
            yield queue1.pop()
        if queue2:
            yield queue2.pop()

# Пример использования
queue1 = deque([1, 2, 3])
queue2 = deque([4, 5, 6])

gen = deque_value_generator(queue1, queue2)
for val in gen:
    print(val)

3
6
2
5
1
4


3)Реализовать классы с методом action():
  а) Класс Pump - в методе action() извлекает очередное значение из генератора и помещает значение в очередь (очередь передается в конструктор).


In [5]:
class Pump:
    def __init__(self, queue, value_generator):
        self.queue = queue
        self.value_generator = value_generator

    def action(self):
        try:
            value = next(self.value_generator)
            self.queue.append(value)
        except StopIteration:
            print("Value generator is empty.")

# Пример использования
from collections import deque

value_generator = iter([1, 2, 3, 4, 5])
queue = deque()

pump = Pump(queue, value_generator)
pump.action()
print(queue)

pump.action()
print(queue)

pump.action()
print(queue)

pump.action()
print(queue)

pump.action()

pump.action()
print(queue)


deque([1])
deque([1, 2])
deque([1, 2, 3])
deque([1, 2, 3, 4])
Value generator is empty.
deque([1, 2, 3, 4, 5])


b) Класс MultiAction - при вызове метода action() n раз вызвает метод action() класса, переданного в конструкторе. Число n также определяется в конструкторе.

In [9]:
class MultiAction:
    def __init__(self, action_class, n):
        self.action_class = action_class
        self.n = n

    def action(self):
        for _ in range(self.n):
            self.action_class.action()

# Пример использования
class SomeClass:
    @staticmethod
    def action():
        print(2+2)

multi_action = MultiAction(SomeClass, 2)
multi_action.action()

4
4


c) Класс MultiPump - в методе action() извлекает очередное значение из генератора и помещает значение в одну из очередей (очереди передается в конструкторе); очереди , в которые помещаются очередные значения, меняются по порядку.


In [10]:
from collections import deque

class MultiPump:
    def __init__(self, queues, value_generator):
        self.queues = queues
        self.value_generator = value_generator
        self.current_queue_index = 0

    def action(self):
        try:
            value = next(self.value_generator)
            self.queues[self.current_queue_index].append(value)
            self.current_queue_index = (self.current_queue_index + 1) % len(self.queues)
        except StopIteration:
            print("Value generator is empty.")

# Пример использования
queues = [deque(), deque(), deque()]
value_generator = iter([1, 2, 3, 4, 5, 6, 7])

multi_pump = MultiPump(queues, value_generator)
for _ in range(7):
    multi_pump.action()

print(queues[0])
print(queues[1])
print(queues[2])


deque([1, 4, 7])
deque([2, 5])
deque([3, 6])


4)При помощи GenFromQ, Pump реализовать систему обработки сообщений. Сообщения создаются генератором сообщений возвращающим случайным образорм одно из сообщений. Сообщения из трех генераторов заканчиваются в три очереди при помощи классов Pump, далее при помощи GenFromQ и Pump объединяются в одну очередь и выводятся на экран (можно реализовать при помощи класса с action и вызываемого при помощи MultiPump).

In [11]:
import random
from collections import deque #Импортируется модуль random и класс deque из модуля collections

class GenFromQ: #Определяется класс GenFromQ, который содержит метод init (конструктор) и метод action
    def __init__(self, queues):
        self.queues = queues #сюда мы передаём наши очереди

    def action(self, output_queue):
        for queue in self.queues:
            while queue:
                output_queue.append(queue.popleft()) #основное действие класса, которое добавляет в итоговую очередь удаленный левый элемент неосновной очереди
class Pump: # Определяется класс Pump, который содержит метод init (конструктор) и метод action
    def __init__(self, queue):
        self.queue = queue # сюда мы передаём очередь, с которой будем дальше работать

    def action(self, gen): #основное действие класса, которое извлекает значение и добавляет его в очередь
        try:
            value = next(gen)
            self.queue.append(value)
        except StopIteration:
            pass

# создаём функцию message_generator, которая возвращает случайное сообщение из списка
def message_generator():
    messages = ["Hi", "How are you?", "I'm glad to see you"]
    while True:
        yield random.choice(messages)

# создаём три очереди и итоговую очередь вывода, имеющие класс deque из модуля collections
queue1 = deque()
queue2 = deque()
queue3 = deque()
output_queue = deque()

# Создаем экземпляры классов Pump для каждой очереди
pump1 = Pump(queue1)
pump2 = Pump(queue2)
pump3 = Pump(queue3)

# Создаем экземпляр класса GenFromQ для объединения сообщений из разных очередей
gen_from_q = GenFromQ([queue1, queue2, queue3])

# Создаем генератор сообщений
gen = message_generator()

# Используя метод action у каждого pump передается gen в качетсве параметра, чтобы загружалось одно случайное сообщене в очередь
pump1.action(gen)
pump2.action(gen)
pump3.action(gen)

# Вызываем метод action у gen_from_q, который извлекает сообщения из всех трех очередей и добавляет их в output_queue
gen_from_q.action(output_queue)

# В цикле печатаются все сообщения из итоговой очереди с помощью метода popleft, пока очередь не станет пустой
while output_queue:
    print(output_queue.popleft())

Hi
I'm glad to see you
How are you?
